## Tutorial from https://github.com/shaohua0116/VAE-Tensorflow

In [84]:
import time
import importlib
import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim import fully_connected as fc
import matplotlib.pyplot as plt 
import glob
from collections import OrderedDict
%matplotlib inline
from datetime import datetime
import os
from pathlib import Path
import layer_def as ld
importlib.reload(ld)

<module 'layer_def' from '/Users/gongbing/PycharmProjects/GAN_practice/layer_def.py'>

In [85]:
help(ld.transpose_conv_layer)

Help on function transpose_conv_layer in module layer_def:

transpose_conv_layer(inputs, kernel_size, stride, num_features, idx, activate='relu')



In [86]:
train_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train"
test_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/test"
val_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train"

In [87]:
num_epochs = 50
batch_size=40
input_dim=3
num_sample = 1500
def make_dataset(type="train"):
    if type=="train": filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train/*.tfrecords")
    if type=="val":filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/val/*.tfrecords")
    if type=="test":filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/test/*.tfrecords")
    
    def parser(serialized_example):
            seqs = OrderedDict()
            keys_to_features = {
                # 'width': tf.FixedLenFeature([], tf.int64),
                # 'height': tf.FixedLenFeature([], tf.int64),
                'sequence_length': tf.FixedLenFeature([], tf.int64),
                # 'channels': tf.FixedLenFeature([],tf.int64),
                # 'images/encoded':  tf.FixedLenFeature([], tf.string)
                'images/encoded': tf.VarLenFeature(tf.float32)
            }
            parsed_features = tf.parse_single_example(serialized_example, keys_to_features)
            seq = tf.sparse_tensor_to_dense(parsed_features["images/encoded"])
            print("Seq= ",seq.shape)
            images = tf.reshape(seq, [20,64, 64,3], name = "reshape_new")
            seqs["images"] = images
            return seqs
    dataset = tf.data.TFRecordDataset(filenames, buffer_size = 8 * 1024 * 1024)
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.apply(tf.contrib.data.map_and_batch(
            parser, batch_size, drop_remainder = True, num_parallel_calls = None))
    #dataset = dataset.map(parser)
    # num_parallel_calls = None if shuffle else 1  # for reproducibility (e.g. sampled subclips from the test set)
    # dataset = dataset.apply(tf.contrib.data.map_and_batch(
    #    _parser, batch_size, drop_remainder=True, num_parallel_calls=num_parallel_calls)) #  Bing: Parallel data mapping, num_parallel_calls normally depends on the hardware, however, normally should be equal to be the usalbe number of CPUs
    dataset = dataset.prefetch(batch_size)  # Bing: Take the data to buffer inorder to save the waiting time for GPU
    print("dataset",dataset)
    #dataset = dataset.repeat(max_step)
    #dataset = dataset.batch(batch_size)
    #iterator = dataset.make_one_shot_iterator() #One shot iterator will pool all the data once and memery issue
    iterator = dataset.make_initializable_iterator()
    return iterator

class VariantionalAutoencoder(object):

    def __init__(self, learning_rate=1e-4, batch_size=64, n_z=16):
        # Set hyperparameters
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_z = n_z
        # Build the graph
        self.build()
        # Initialize paramters
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
        # Summary op
        self.loss_summary = tf.summary.scalar("recon_losses", self.recon_loss)
        self.loss_summary = tf.summary.scalar("latent_losses", self.latent_loss)
        self.summary_op = tf.summary.merge_all()
        self.summary_dir = "./"
        self.base_dir = "VAE"
        self.checkpoint_dir =  "VAE" + "/checkpoint"
        Path(self.checkpoint_dir).mkdir(parents=True, exist_ok=True)
        self.train_log_file = self.base_dir + "/train_" + datetime.now().strftime("%Y%m%d-%H%M%S")
        self.val_log_file = self.base_dir + "/val_" + datetime.now().strftime("%Y%m%d-%H%M%S")
        self.train_writer = tf.summary.FileWriter(self.train_log_file, self.sess.graph)
        self.val_writer = tf.summary.FileWriter(self.val_log_file, self.sess.graph)

         
    def vae_arc1(self):
        
        # Encode
        # x -> z_mean, z_sigma -> z
        f1 = fc(self.x, 128, scope='enc_fc1', activation_fn=tf.nn.relu)
        f2 = fc(f1, 64, scope='enc_fc2', activation_fn=tf.nn.relu)
        f3 = fc(f2, 32, scope='enc_fc3', activation_fn=tf.nn.relu)
        self.z_mu = fc(f3, self.n_z, scope='enc_fc4_mu', 
                       activation_fn=None)
        self.z_log_sigma_sq = fc(f3, self.n_z, scope='enc_fc4_sigma', 
                                 activation_fn=None)
        eps = tf.random_normal(shape=tf.shape(self.z_log_sigma_sq),mean=0, stddev=1, dtype=tf.float32)
        
        self.z = self.z_mu + tf.sqrt(tf.exp(self.z_log_sigma_sq)) * eps

        # Decode
        # z -> x_hat
        g1 = fc(self.z, 32, scope='dec_fc1', activation_fn=tf.nn.relu)
        g2 = fc(g1, 64, scope='dec_fc2', activation_fn=tf.nn.relu)
        g3 = fc(g2, 128, scope='dec_fc3', activation_fn=tf.nn.relu)
        self.x_hat = fc(g3, input_dim, scope='dec_fc4', activation_fn=tf.sigmoid)
        return 
    

    def vae_arc2(self):
        
        # Encode
        # x -> z_mean, z_sigma -> z
        f3 = fc(self.x, 32, scope='enc_fc1', activation_fn=tf.nn.relu)

        self.z_mu = fc(f3, self.n_z, scope='enc_fc4_mu', 
                       activation_fn=None)
        self.z_log_sigma_sq = fc(f3, self.n_z, scope='enc_fc4_sigma', 
                                 activation_fn=None)
        eps = tf.random_normal(shape=tf.shape(self.z_log_sigma_sq),mean=0, stddev=1, dtype=tf.float32)
        self.z = self.z_mu + tf.sqrt(tf.exp(self.z_log_sigma_sq)) * eps
        
        # Decode
        # z -> x_hat
        g1 = fc(self.z, 32, scope='dec_fc1', acxtivation_fn=tf.nn.relu)
        self.x_hat = fc(g1, input_dim, scope='dec_fc4', activation_fn=tf.sigmoid)
        return 
    
    def mymodel(self):
        pass
        
    @staticmethod
    def vae_arc3(x, l_name=0):
        seq_name = "sq_" + str(l_name) + "_"
        print ("DBBUG: INPUT",x)
        conv1 = ld.conv_layer(x, 3, 2, 8, seq_name + "encode_1")
        print("Encode_1_shape",conv1.shape) #(?,2,2,8)
        # conv2
        conv2 = ld.conv_layer(conv1, 3, 1, 8, seq_name +"encode_2") #(?,2,2,8)
        print("Encode 2_shape,",conv2.shape)
        # conv3
        conv3 = ld.conv_layer(conv2, 3, 2, 8, seq_name +"encode_3") # (?,1,1,8)
        print("Encode 3_shape, ", conv3.shape)
        #flatten
        conv4 = tf.layers.Flatten()(conv3)
        print("Encode 4_shape, ", conv4.shape)
        #Todo: to conv3 to z 
        z_mu = ld.fc_layer(conv4, hiddens=16, idx= seq_name + "enc_fc4_m")
        z_log_sigma_sq = ld.fc_layer(conv4, hiddens=16, idx= seq_name + "enc_fc4_m"'enc_fc4_sigma')        
        eps = tf.random_normal(shape=tf.shape(z_log_sigma_sq), mean=0, stddev=1, dtype=tf.float32)
        z = z_mu + tf.sqrt(tf.exp(z_log_sigma_sq)) * eps
        print("latend variables z ", z)
        z2 = ld.fc_layer(z, hiddens=16*16*8, idx= seq_name +"deenc_fc1")
        print("latend variables z2 ", z2)
        z3 = tf.reshape(z2,[-1,16,16,8])
        print("latend variables z3 ", z3) 
        # conv5
        conv5 = ld.transpose_conv_layer(z3, 3, 2, 8, seq_name +"decode_5") #(16,1,1,8)inputs, kernel_size, stride, num_features
        print("Decode 5 shape",conv5.shape)
        conv6 = ld.transpose_conv_layer(z3, 3, 4, 8, seq_name +"decode_6") #(16,1,1,8)inputs, kernel_size, stride, num_features
        print("Decode 6 shape",conv6.shape)
        # conv7
        #conv7 = ld.transpose_conv_layer(conv6, 2, 1, 8, "decode_7") #(16,2,2,8)
        #conv7 = ld.transpose_conv_layer(conv6, 3, 1, 8, "decode_7")
        #print("Decode 7 shape", conv7.shape)
        # x_1
        x_hat = ld.transpose_conv_layer(conv6, 2, 1, 3, seq_name +"decode_8")  # set activation to linear
        print("X_hat", x_hat.shape)
        return x_hat, z_mu,z_log_sigma_sq, z
    
    def vae_arc_all(self):
        X = []
        z_log_sigma_sq_all = []
        z_mu_all = []
        for i in range(20):
            q, z_mu, z_log_sigma_sq, z = VariantionalAutoencoder.vae_arc3(self.x[:,i,:,:,:], l_name=i)
            X.append(q)
            z_log_sigma_sq_all.append(z_log_sigma_sq)
            z_mu_all.append(z_mu)
        x_hat = tf.stack(X,axis = 1)
        z_log_sigma_sq_all = tf.stack(z_log_sigma_sq_all,axis = 1)
        z_mu_all = tf.stack(z_mu_all,axis = 1)
        print ("X_hat",x_hat.shape)
        print ("zlog_sigma_sq_all",z_log_sigma_sq_all.shape)
        return x_hat,z_log_sigma_sq_all,z_mu_all
        
        
    # Build the netowrk and the loss functions
    def build(self):
        tf.reset_default_graph()
        self.train_iterator = make_dataset(type="train")
        self.val_iterator = make_dataset(type="val")
        self.test_iterator = make_dataset(type="test")
        self.x = tf.placeholder(tf.float32, [None,20,64,64,3])
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        self.increment_global_step = tf.assign_add(self.global_step,1,name = 'increment_global_step')
        
        
        #ARCHITECTURE
        #self.mymodel()
        
        self.x_hat,self.z_log_sigma_sq,self.z_mu = self.vae_arc_all()
        # Loss
        # Reconstruction loss
        # Minimize the cross-entropy loss
#         epsilon = 1e-10
#         recon_loss = -tf.reduce_sum(
#             self.x[:,1:,:,:,:] * tf.log(epsilon+self.x_hat[:,:-1,:,:,:]) + 
#             (1-self.x[:,1:,:,:,:]) * tf.log(epsilon+1-self.x_hat[:,:-1,:,:,:]), 
#             axis=1
#         )

#        self.recon_loss = tf.reduce_mean(recon_loss)
        self.recon_loss = tf.reduce_mean(tf.square(self.x[:,1:,:,:,0]  - self.x_hat[:,:-1,:,:,0]))

        # Latent loss
        # KL divergence: measure the difference between two distributions
        # Here we measure the divergence between 
        # the latent distribution and N(0, 1)
        latent_loss = -0.5 * tf.reduce_sum(
            1 + self.z_log_sigma_sq - tf.square(self.z_mu) - 
            tf.exp(self.z_log_sigma_sq), axis=1)
        self.latent_loss = tf.reduce_mean(latent_loss)

        self.total_loss = self.recon_loss + self.latent_loss
        self.train_op = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.total_loss,global_step=self.global_step)
        
        # Build a saver
        self.saver = tf.train.Saver(tf.global_variables())
        
        self.losses = {
            'recon_loss': self.recon_loss,
            'latent_loss': self.latent_loss,
            'total_loss': self.total_loss,
        }      # H(x, x_hat) = -\Sigma x*log(x_hat) + (1-x)*log(1-x_hat)

        
        #self.ckpt = tf.train.Checkpoint(model=self.vae_arc2())
        #self.manager = tf.train.CheckpointManager(self.ckpt,self.checkpoint_dir,max_to_keep=3)
        return

    # Execute the forward and the backward pass
    def run_single_step(self):
        global_step = self.sess.run(self.global_step)
        try:
            train_batch = self.sess.run(self.train_iterator.get_next())
            print("Train_batch shape", train_batch["images"].shape)
            x_hat, train_summary, _, train_losses = self.sess.run([self.x_hat, self.summary_op, self.train_op, self.losses], feed_dict={self.x: train_batch["images"]})
            self.train_writer.add_summary(train_summary, global_step)
            print("x_hat.shape", x_hat.shape)
        except tf.errors.OutOfRangeError:
            print("train out of range error")
        
        try:
            val_batch = self.sess.run(self.val_iterator.get_next())
            val_summary, _, val_losses = self.sess.run([self.summary_op,self.train_op, self.losses], feed_dict={self.x: val_batch["images"]})
            self.val_writer.add_summary(val_summary, global_step)
        except tf.errors.OutOfRangeError:
            print("train out of range error")
        
        return train_losses,val_losses

    # x -> x_hat
    def reconstructor(self, x):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.x: x})
        return x_hat

    # z -> x
    def generator(self, z):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.z: z})
        return x_hat
    
    
    # x -> z
    def transformer(self, x):
        z = self.sess.run(self.z, feed_dict={self.x: x})
        return z


In [81]:
def trainer(model_class, learning_rate=1e-4, 
            batch_size=64, num_epoch=100, n_z=16, log_step=5):
    # Create a model    
    model = model_class(learning_rate=learning_rate, batch_size=batch_size, n_z=n_z)

    # Training loop    
    for epoch in range(num_epoch):
        start_time = time.time()
        # Run an epoch
        for iter in range(num_sample // batch_size):
            # Get a batch
            step = epoch*(num_sample // batch_size) +  iter
            train_losses,val_losses = model.run_single_step()
            print ("Train_loss: {}; Val_loss{}".format(train_losses,val_losses))
            checkpoint_path = os.path.join(model.checkpoint_dir, 'model.ckpt')
            model.saver.save(model.sess, model.checkpoint_path, global_step =step)
        end_time = time.time()
        
        # Log the loss
#         if epoch % log_step == 0:
#             log_str = '[Epoch {}] '.format(epoch)
#             for k, v in self.recon_loss.items():
#                 log_str += '{}: {:.3f}  '.format(k, v)
#             log_str += '({:.3f} sec/epoch)'.format(end_time - start_time)
#             print(log_str)

    print('Done!')
    return model

In [ ]:
model_vae = trainer(VariantionalAutoencoder)

In [ ]:
%load_ext tensorboard.notebook
%tensorboard --logdir=./ --host localhost

In [88]:
def trainer_and_checkpoint(model_class, learning_rate=1e-4, 
            batch_size=64, num_epoch=100, n_z=16, log_step=5):
    #restore the existing checkpoints  
    
    model = model_class(learning_rate=learning_rate, batch_size=batch_size, n_z=n_z)
    ckpt = tf.train.get_checkpoint_state(model.checkpoint_dir)
    
    if ckpt and ckpt.model_checkpoint_path:
        #Extract from checkpoint filename
        global_step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
        sess = tf.Session()
        print("Restore from {}".format(ckpt.model_checkpoint_path))
        graph = tf.get_default_graph()
        saver = tf.train.Saver(tf.global_variables())
        #saver = tf.train.import_meta_graph(os.path.join(model.checkpoint_dir,'model.ckpt-{}.meta'.format(global_step)))
        saver.restore(sess,tf.train.latest_checkpoint(model.checkpoint_dir))
        loaded_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=tf.get_variable_scope().name)
        print("loaded_vars",loaded_vars)
        
##        The following for extract global step does not work, it always reset to 0 
#         increment_global_step = graph.get_tensor_by_name("increment_global_step:0").eval()
#         other_var = graph.get_tensor_by_name("enc_fc1/weights:0").eval()
#         print ("Recover global step",increment_global_step)
#         print ("other var",other_var)
    else:
        print("Initializer from scratch")
        global_step = model.sess.run(model.global_step)
    # Training loop    
    for epoch in range(num_epoch):
        model.sess.run(model.train_iterator.initializer)
        model.sess.run(model.val_iterator.initializer)
        start_time = time.time()
        # Run an epoch
        for iter in range(num_sample // batch_size):
            # Get a batch
            #step = epoch*(num_sample // batch_size) +  iter
            print ("global step",model.global_step)
            train_losses,val_losses = model.run_single_step()
            print ("Train_loss: {}; Val_loss{} for global step {}".format(train_losses,val_losses,global_step))
            checkpoint_path = os.path.join(model.checkpoint_dir, 'model_arc4.ckpt')
            model.saver.save(model.sess,checkpoint_path, global_step = global_step)
            global_step = global_step  +1
        end_time = time.time()
        
        # Log the loss
#         if epoch % log_step == 0:
#             log_str = '[Epoch {}] '.format(epoch)
#             for k, v in self.recon_loss.items():
#                 log_str += '{}: {:.3f}  '.format(k, v)
#             log_str += '({:.3f} sec/epoch)'.format(end_time - start_time)
#             print(log_str)
    print('Done!')
    return model    
    

In [89]:
model_vae2 = trainer_and_checkpoint(VariantionalAutoencoder)

Seq=  (?,)
dataset <DatasetV1Adapter shapes: OrderedDict([(images, (40, 20, 64, 64, 3))]), types: OrderedDict([(images, tf.float32)])>
Seq=  (?,)
dataset <DatasetV1Adapter shapes: OrderedDict([(images, (40, 20, 64, 64, 3))]), types: OrderedDict([(images, tf.float32)])>
Seq=  (?,)
dataset <DatasetV1Adapter shapes: OrderedDict([(images, (40, 20, 64, 64, 3))]), types: OrderedDict([(images, tf.float32)])>
DBBUG: INPUT Tensor("strided_slice:0", shape=(?, 64, 64, 3), dtype=float32)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 3_shape,  (?, 16, 16, 8)
Encode 4_shape,  (?, 2048)
latend variables z  Tensor("add:0", shape=(?, 16), dtype=float32)
latend variables z2  Tensor("sq_0_deenc_fc1_fc/Relu:0", shape=(?, 2048), dtype=float32)
latend variables z3  Tensor("Reshape:0

X_hat (?, 64, 64, 3)
DBBUG: INPUT Tensor("strided_slice_8:0", shape=(?, 64, 64, 3), dtype=float32)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 3_shape,  (?, 16, 16, 8)
Encode 4_shape,  (?, 2048)
latend variables z  Tensor("add_8:0", shape=(?, 16), dtype=float32)
latend variables z2  Tensor("sq_8_deenc_fc1_fc/Relu:0", shape=(?, 2048), dtype=float32)
latend variables z3  Tensor("Reshape_8:0", shape=(?, 16, 16, 8), dtype=float32)
input_channel 8
output_shape Tensor("sq_8_decode_5_trans_conv/stack:0", shape=(4,), dtype=int32)
Decode 5 shape (?, 32, 32, 8)
input_channel 8
output_shape Tensor("sq_8_decode_6_trans_conv/stack:0", shape=(4,), dtype=int32)
Decode 6 shape (?, 64, 64, 8)
input_channel 8
output_shape Tensor("sq_8_decode_8_trans_conv/stack:0", shape=(4,), 

output_shape Tensor("sq_16_decode_8_trans_conv/stack:0", shape=(4,), dtype=int32)
X_hat (?, 64, 64, 3)
DBBUG: INPUT Tensor("strided_slice_17:0", shape=(?, 64, 64, 3), dtype=float32)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 3_shape,  (?, 16, 16, 8)
Encode 4_shape,  (?, 2048)
latend variables z  Tensor("add_17:0", shape=(?, 16), dtype=float32)
latend variables z2  Tensor("sq_17_deenc_fc1_fc/Relu:0", shape=(?, 2048), dtype=float32)
latend variables z3  Tensor("Reshape_17:0", shape=(?, 16, 16, 8), dtype=float32)
input_channel 8
output_shape Tensor("sq_17_decode_5_trans_conv/stack:0", shape=(4,), dtype=int32)
Decode 5 shape (?, 32, 32, 8)
input_channel 8
output_shape Tensor("sq_17_decode_6_trans_conv/stack:0", shape=(4,), dtype=int32)
Decode 6 shape (?, 64, 64,

global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.14134276, 'latent_loss': 0.004266409, 'total_loss': 0.14560917}; Val_loss{'recon_loss': 0.19403718, 'latent_loss': 0.003782743, 'total_loss': 0.19781993} for global step 15
global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.14720733, 'latent_loss': 0.0033752532, 'total_loss': 0.15058258}; Val_loss{'recon_loss': 0.24069917, 'latent_loss': 0.0030258459, 'total_loss': 0.24372502} for global step 16
global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.15892862, 'latent_loss': 0.0027190535, 'total_loss': 0.16164768}; Val_loss{'recon_loss': 0.24238637, 'latent_loss': 0.002446074, 'total_loss': 0.24483244}

Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.14486207, 'latent_loss': 0.0, 'total_loss': 0.14486207}; Val_loss{'recon_loss': 0.23565786, 'latent_loss': 0.0, 'total_loss': 0.23565786} for global step 39
global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.15616074, 'latent_loss': 0.0, 'total_loss': 0.15616074}; Val_loss{'recon_loss': 0.23732272, 'latent_loss': 0.0, 'total_loss': 0.23732272} for global step 40
global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.15732256, 'latent_loss': 0.0, 'total_loss': 0.15732256}; Val_loss{'recon_loss': 0.2243969, 'latent_loss': 0.0, 'total_loss': 0.2243969} for global step 41
global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64

global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.11904074, 'latent_loss': 0.0, 'total_loss': 0.11904074}; Val_loss{'recon_loss': 0.1977042, 'latent_loss': 0.0, 'total_loss': 0.1977042} for global step 65
global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.10830036, 'latent_loss': 0.0, 'total_loss': 0.10830036}; Val_loss{'recon_loss': 0.18258892, 'latent_loss': 0.0, 'total_loss': 0.18258892} for global step 66
global step <tf.Variable 'global_step:0' shape=() dtype=int32_ref>
Train_batch shape (40, 20, 64, 64, 3)
x_hat.shape (40, 20, 64, 64, 3)
Train_loss: {'recon_loss': 0.10661815, 'latent_loss': 0.0, 'total_loss': 0.10661815}; Val_loss{'recon_loss': 0.1779859, 'latent_loss': 0.0, 'total_loss': 0.1779859} for global step 67
global step <tf.Variable 'global_st

KeyboardInterrupt: 

In [90]:
#def model_prediction():
model = VariantionalAutoencoder(learning_rate=1e-4, batch_size=64, n_z=16)
ckpt = tf.train.get_checkpoint_state(model.checkpoint_dir)
global_step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
#First let's load meta graph and restore weights
sess = tf.Session()  
saver = tf.train.Saver(tf.global_variables())
saver.restore(sess,tf.train.latest_checkpoint(model.checkpoint_dir))
#latest_checkpoints = saver.restore(sess,tf.train.latest_checkpoint('./'))
graph = tf.get_default_graph()
#op = sess.graph.get_operations()
#graph.get_tensor_by_name("dec_fc4")
loaded_vars = tf.trainable_variables() 
#loaded_vars
#op_to_restore = graph.get_tensor_by_name("dec_fc4/biases:0")
test_iterator = make_dataset(type="test")
sess.run(test_iterator.initializer)
for i in range(1):
    test_batch = sess.run(test_iterator.get_next())
    # print("test_batch",test_batch["images"].shape)
    #op_to_restore.eval(feed_dict={model.x: test_batch["images"]})
    predict_images,  recon_loss = sess.run([model.x_hat, model.recon_loss], feed_dict={model.x: test_batch["images"]})
    #plot real and predicted images
    real_img = test_batch["images"][0]
    pred_img = predict_images[0]
    #plt.figure(figsize=(8, 8))
    #plt.imshow(real_img[0])
    print("recon_loss",recon_loss)
    print("real_image",real_img.shape)
    print("predic image", pred_img.shape)
#https://jhui.github.io/2017/03/08/TensorFlow-variable-sharing/

Seq=  (?,)
dataset <DatasetV1Adapter shapes: OrderedDict([(images, (40, 20, 64, 64, 3))]), types: OrderedDict([(images, tf.float32)])>
Seq=  (?,)
dataset <DatasetV1Adapter shapes: OrderedDict([(images, (40, 20, 64, 64, 3))]), types: OrderedDict([(images, tf.float32)])>
Seq=  (?,)
dataset <DatasetV1Adapter shapes: OrderedDict([(images, (40, 20, 64, 64, 3))]), types: OrderedDict([(images, tf.float32)])>
DBBUG: INPUT Tensor("strided_slice:0", shape=(?, 64, 64, 3), dtype=float32)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 3_shape,  (?, 16, 16, 8)
Encode 4_shape,  (?, 2048)
latend variables z  Tensor("add:0", shape=(?, 16), dtype=float32)
latend variables z2  Tensor("sq_0_deenc_fc1_fc/Relu:0", shape=(?, 2048), dtype=float32)
latend variables z3  Tensor("Reshape:0

Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 3_shape,  (?, 16, 16, 8)
Encode 4_shape,  (?, 2048)
latend variables z  Tensor("add_8:0", shape=(?, 16), dtype=float32)
latend variables z2  Tensor("sq_8_deenc_fc1_fc/Relu:0", shape=(?, 2048), dtype=float32)
latend variables z3  Tensor("Reshape_8:0", shape=(?, 16, 16, 8), dtype=float32)
input_channel 8
output_shape Tensor("sq_8_decode_5_trans_conv/stack:0", shape=(4,), dtype=int32)
Decode 5 shape (?, 32, 32, 8)
input_channel 8
output_shape Tensor("sq_8_decode_6_trans_conv/stack:0", shape=(4,), dtype=int32)
Decode 6 shape (?, 64, 64, 8)
input_channel 8
output_shape Tensor("sq_8_decode_8_trans_conv/stack:0", shape=(4,), dtype=int32)
X_hat (?, 64, 64, 3)
DBBUG: INPUT Tensor("strided_slice_9:0", shape=(?, 64, 64, 3), dtype=float32)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?

output_shape Tensor("sq_16_decode_8_trans_conv/stack:0", shape=(4,), dtype=int32)
X_hat (?, 64, 64, 3)
DBBUG: INPUT Tensor("strided_slice_17:0", shape=(?, 64, 64, 3), dtype=float32)
conv_layer activation function relu
DEBUG input shape (?, 64, 64, 3)
Encode_1_shape (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 2_shape, (?, 32, 32, 8)
conv_layer activation function relu
DEBUG input shape (?, 32, 32, 8)
Encode 3_shape,  (?, 16, 16, 8)
Encode 4_shape,  (?, 2048)
latend variables z  Tensor("add_17:0", shape=(?, 16), dtype=float32)
latend variables z2  Tensor("sq_17_deenc_fc1_fc/Relu:0", shape=(?, 2048), dtype=float32)
latend variables z3  Tensor("Reshape_17:0", shape=(?, 16, 16, 8), dtype=float32)
input_channel 8
output_shape Tensor("sq_17_decode_5_trans_conv/stack:0", shape=(4,), dtype=int32)
Decode 5 shape (?, 32, 32, 8)
input_channel 8
output_shape Tensor("sq_17_decode_6_trans_conv/stack:0", shape=(4,), dtype=int32)
Decode 6 shape (?, 64, 64,

In [91]:
list(real_img[0][:,:,0])

[array([0.5560357 , 0.5554696 , 0.5550167 , 0.55508465, 0.55508465,
        0.55490345, 0.5546091 , 0.55422413, 0.55320513, 0.5519144 ,
        0.5500576 , 0.5483366 , 0.54668355, 0.54464555, 0.54299253,
        0.54215467, 0.54122627, 0.54025257, 0.5384863 , 0.5368785 ,
        0.53554255, 0.5287492 , 0.5191933 , 0.512717  , 0.45368317,
        0.4614049 , 0.4616087 , 0.45902723, 0.46871904, 0.47924864,
        0.49399012, 0.49879074, 0.5079164 , 0.5121962 , 0.51584196,
        0.51824224, 0.51969147, 0.52066517, 0.51609105, 0.51217353,
        0.5048821 , 0.48690245, 0.47519532, 0.46878695, 0.46280885,
        0.45766857, 0.452166  , 0.44648227, 0.44734275, 0.44849762,
        0.46081614, 0.44849762, 0.4451236 , 0.43957573, 0.4354318 ,
        0.43414107, 0.4350695 , 0.43713015, 0.44000596, 0.45092055,
        0.4548154 , 0.45830262, 0.4615634 , 0.46244654], dtype=float32),
 array([0.55576396, 0.5558998 , 0.55601305, 0.5557413 , 0.5555602 ,
        0.5554922 , 0.5548582 , 0.5539071 ,

In [96]:
list(pred_img[0][:,:,0])

[array([0.60583067, 0.5260038 , 0.5254352 , 0.52581936, 0.52461094,
        0.52681446, 0.52753824, 0.5257369 , 0.5257905 , 0.5294059 ,
        0.5310618 , 0.5272046 , 0.52880555, 0.529532  , 0.52752566,
        0.5266587 , 0.52646095, 0.5261761 , 0.5289267 , 0.5248511 ,
        0.5280221 , 0.527828  , 0.5307122 , 0.52752084, 0.5275937 ,
        0.526298  , 0.53089595, 0.5263666 , 0.5290129 , 0.5276999 ,
        0.53065884, 0.524562  , 0.52875054, 0.5279408 , 0.52553576,
        0.52646583, 0.52612406, 0.52639866, 0.52589744, 0.5256294 ,
        0.52622545, 0.5260795 , 0.5287689 , 0.5257502 , 0.5259412 ,
        0.52692443, 0.5260022 , 0.52579075, 0.5261981 , 0.5259677 ,
        0.52737284, 0.5258965 , 0.5264288 , 0.5251645 , 0.5281703 ,
        0.5246728 , 0.5274922 , 0.5259515 , 0.5277095 , 0.5239311 ,
        0.52596587, 0.52565426, 0.52816695, 0.5264233 ], dtype=float32),
 array([0.60209954, 0.5767369 , 0.57665586, 0.57688797, 0.5775436 ,
        0.57643133, 0.5745925 , 0.5756515 ,

In [ ]:
fig = plt.figure(figsize=(18,6))
    gs = gridspec.GridSpec(1, 10)
    gs.update(wspace = 0., hspace = 0.)
    ts = [0,5,9,10,12,14,16,18,19]
    xlables = [round(i,2) for i in list(np.linspace(np.min(lon),np.max(lon),5))]
    ylabels = [round(i,2) for i  in list(np.linspace(np.max(lat),np.min(lat),5))] 
    for t in range(len(ts)):
        #if t==0 : ax1=plt.subplot(gs[t])
        ax1 = plt.subplot(gs[t])
        input_image = input_images_[ts[t], :, :, 0] * (321.46630859375 - 235.2141571044922) + 235.2141571044922
        plt.imshow(input_image, cmap = 'jet', vmin=270, vmax=300)
        ax1.title.set_text("t = " + str(ts[t]+1))
        plt.setp([ax1], xticks = [], xticklabels = [], yticks = [], yticklabels = [])
        if t == 0:
            plt.setp([ax1], xticks = list(np.linspace(0, 64, 3)), xticklabels = xlables, yticks = list(np.linspace(0, 64, 3)), yticklabels = ylabels)
            plt.ylabel("Ground Truth", fontsize=10)
    plt.savefig(os.path.join(args.output_png_dir, "Ground_Truth_Sample_" + str(name) + ".jpg"))
    plt.clf()

In [2]:
plt.figure(figsize=(8, 8))
plt.imshow(real_img[:,:,0])

NameError: name 'plt' is not defined

In [6]:
np.array(real_img).shape

(1, 3, 3)